# Installation (only required for the first run)

In [1]:
!git clone --recursive https://github.com/kwea123/nerf_pl

%cd /content/nerf_pl
!pip install -r requirements.txt

%cd /content/nerf_pl/torchsearchsorted
!pip install .

fatal: destination path 'nerf_pl' already exists and is not an empty directory.
/content/nerf_pl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/nerf_pl/torchsearchsorted
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/nerf_pl/torchsearchsorted
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for torchsearchsorted: filename=torchsearchsorted-1.1-cp38-cp38-linux_x86_64.whl size=3223661 sha256=1e2fbdee8db31c8cc0011ea136b536a8f554c5531d47c84dc405a3227eee322f
  Stored in directory: /tmp/pip-ephem-wheel-ca

# Mount your google drive (to access data)

In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# Train! (depending on number of epochs, takes about 5~8 hours)

### model weights are saved to `ckpts/$EXP`
### training logs (loss/PSNR evolution) are saved to `logs/$EXP`

## Forward facing scene

In [6]:
%cd /content/nerf_pl

import os
# set training configurations here
os.environ['ROOT_DIR'] = "/content/drive/My Drive/myNeRF/hall"
                         # directory containing the data
os.environ['IMG_W'] = "136" # image width (do not set too large)
os.environ['IMG_H'] = "102" # image height (do not set too large)
os.environ['NUM_EPOCHS'] = "20" # number of epochs to train (depending on how many images there are,
                                # 20~30 might be enough)
os.environ['EXP'] = "hall" # name of the experience (arbitrary)

!python train.py \
   --dataset_name llff \
   --root_dir "$ROOT_DIR" \
   --N_importance 64 --img_wh $IMG_W $IMG_H \
   --num_epochs $NUM_EPOCHS --batch_size 1024 \
   --optimizer adam --lr 5e-4 \
   --lr_scheduler cosine \
   --exp_name $EXP

/content/nerf_pl
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
Traceback (most recent call last):
  File "train.py", line 180, in <module>
    trainer.fit(system)
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py", line 730, in fit
    model.prepare_data()
  File "train.py", line 80, in prepare_data
    self.train_dataset = dataset(split='train', **kwargs)
  File "/content/nerf_pl/datasets/llff.py", line 173, in __init__
    self.read_meta()
  File "/content/nerf_pl/datasets/llff.py", line 182, in read_meta
    assert len(poses_bounds) == len(self.image_paths), \
AssertionError: Mismatch between number of images and number of poses! Please rerun COLMAP!


## 360 inward-facing scene

In [15]:
%cd /content/nerf_pl

import os
# set training configurations here
os.environ['ROOT_DIR'] = "/content/drive/My Drive/myNeRF/Box"
                         # directory containing the data
os.environ['IMG_W'] = "504" # image width (do not set too large)
os.environ['IMG_H'] = "378" # image height (do not set too large)
os.environ['NUM_EPOCHS'] = "1" # number of epochs to train (depending on how many images there are,
                                # 20~30 might be enough)
os.environ['EXP'] = "Box" # name of the experience (arbitrary)

!python train.py \
   --dataset_name llff \
   --root_dir "$ROOT_DIR" \
   --N_importance 64 --img_wh $IMG_W $IMG_H \
   --spheric --use_disp \
   --num_epochs $NUM_EPOCHS --batch_size 1024 \
   --optimizer adam --lr 5e-4 \
   --lr_scheduler cosine \
   --exp_name $EXP

/content/nerf_pl
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
val image is /content/drive/My Drive/myNeRF/Box/images/IMG_2776.JPG
Epoch 1: 100% 4838/4839 [21:51<00:00,  3.69it/s, loss=0.007, train_psnr=24.3, v_num=4]
Validating: 0it [00:00, ?it/s]
Epoch 1: 100% 4839/4839 [22:12<00:00,  3.63it/s, loss=0.007, train_psnr=24.3, v_num=4, val_loss=0.0103, val_psnr=22.9]
Epoch 1: 100% 4839/4839 [22:12<00:00,  3.63it/s, loss=0.007, train_psnr=24.3, v_num=4, val_loss=0.0103, val_psnr=22.9]
INFO:lightning:

Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_train_start      	|  0.0054188      	|  0.0054188      
on_epoch_start      	|  0.0053847      	|  0.0053847      
get_train_batch     	|  0.00044724     	|  2.1642         
on_batch_start      	|  1.0217e-05     	|  0.049431       
model_forward       	|

# Testing! (takes about 20~30 minutes)

### You can also download the pretrained model for `fern` [here](https://github.com/kwea123/nerf_pl/releases)

### The results are saved to `results/llff/$SCENE`

In [ ]:
os.environ['SCENE'] = 'Box'
os.environ['CKPT_PATH'] = '/content/nerf_pl/ckpts/Box/epoch=0.ckpt'

!python eval.py \
   --root_dir "$ROOT_DIR" \
   --dataset_name llff --scene_name $SCENE \
   --img_wh $IMG_W $IMG_H --N_importance 64 --ckpt_path $CKPT_PATH

  2% 3/120 [00:55<36:14, 18.59s/it]